In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD
from torch.utils.data import Dataset, DataLoader

import numpy as np

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [7]:
torch.__version__

'2.3.0'

In [8]:
np.random.seed(123)  

X = np.load('../generated_games/features-200.npy')
Y = np.load('../generated_games/labels-200.npy')
samples = X.shape[0]
size = 9
input_shape = (size, size, 1)  # The input data shape is 3-dimensional, we use one plane of a 9x9 board representation.

X = X.reshape(samples, 1, size, size)  # We then reshape our input data accordingly.

train_samples = 10000
X_train, X_test = X[:train_samples], X[train_samples:]
Y_train, Y_test = Y[:train_samples], Y[train_samples:]

In [9]:
class MCTSAgentCNN(nn.Module):
    def __init__(self, IMG_SIZE=9):
        super(MCTSAgentCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)   # The first layer in our network is a Conv2D layer with 32 output filters.
        self.dropout1 = nn.Dropout(p=0.6)
        self.conv2 = nn.Conv2d(32, 64, 3)  # For this layer we choose a 3 by 3 convolutional kernel.
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.dropout2 = nn.Dropout(0.6)
        self.fc1 = nn.Linear(in_features=64*2*2, out_features=128)
        self.dropout3 = nn.Dropout(0.6)
        self.fc2 = nn.Linear(128, IMG_SIZE*IMG_SIZE)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.dropout2(x)
        x = x.view(-1, 64*2*2)  # We then flatten the 3D output of the previous convolutional layer...
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout3(x)
        x = self.fc2(x)  # ... and follow up with two more dense layers, as we did in the MLP example.
        x = F.softmax(x, dim=-1)
        return x

In [10]:
def compute_acc(argmax, y):
    count = 0
    for i in range(len(argmax)):
        if argmax[i] == y[i]:
            count += 1
    return count / len(argmax)

In [11]:
IMG_SIZE = 9
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NUM_EPOCHES = 5

In [12]:
class GameDataset(Dataset):
    def __init__(self, features, labels, transform=None):
        self.features = features
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx]).float()
        label = torch.tensor(self.labels[idx]).float()

        if self.transform:
            feature = self.transform(feature)
        
        return feature, label

In [13]:
trainset = GameDataset(X_train, Y_train)
testset = GameDataset(X_test, Y_test)

args = {
    'num_workers': 0,
    'batch_size': BATCH_SIZE,
    'shuffle': True,
}

train_loader = DataLoader(trainset, **args)
test_loader = DataLoader(testset, **args)

model = MCTSAgentCNN(IMG_SIZE).cuda()
print(model)

optimizer = SGD(model.parameters())
loss_fn = nn.CrossEntropyLoss()
model.train()

MCTSAgentCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.6, inplace=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2): Dropout(p=0.6, inplace=False)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (dropout3): Dropout(p=0.6, inplace=False)
  (fc2): Linear(in_features=128, out_features=81, bias=True)
)


MCTSAgentCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.6, inplace=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2): Dropout(p=0.6, inplace=False)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (dropout3): Dropout(p=0.6, inplace=False)
  (fc2): Linear(in_features=128, out_features=81, bias=True)
)

In [14]:
for epoch in range(NUM_EPOCHES):
    tot_loss = 0.0

    for x, y in train_loader:
        optimizer.zero_grad()
        x = x.cuda()
        y = y.argmax(dim=-1)
        y_ = model(x)
        loss = loss_fn(y_, y.cuda()) 
        loss.backward()
        tot_loss += loss.item()
        optimizer.step()

    print("Epoch {}, Loss(train) : {}".format(epoch+1, tot_loss))
    if epoch % 2 == 1:
        x, y = next(iter(test_loader))
        x = x.cuda()
        y_ = model(x)
        _, argmax = torch.max(y_, dim=-1)
        _, y_arg = torch.max(y, dim=-1)
        test_acc = compute_acc(argmax, y_arg)

        print("Acc(val) : {}".format(test_acc))


Epoch 1, Loss(train) : 689.9361276626587
Epoch 2, Loss(train) : 689.9338483810425
Acc(val) : 0.0
Epoch 3, Loss(train) : 689.9292593002319
Epoch 4, Loss(train) : 689.9336667060852
Acc(val) : 0.0
Epoch 5, Loss(train) : 689.9312853813171


In [29]:
for x, y in train_loader:
    optimizer.zero_grad()
    x = x.cuda()
    y = y.argmax(dim=-1)
    y_ = model(x)
    loss = loss_fn(y_, y.cuda()) 
    loss.backward()
    tot_loss += loss.item()
    optimizer.step()
    break

In [35]:
y_.shape

torch.Size([64, 81])

In [32]:
y.shape

torch.Size([64])

In [39]:
y_[0]

tensor([0.0087, 0.0154, 0.0127, 0.0095, 0.0131, 0.0103, 0.0123, 0.0099, 0.0148,
        0.0114, 0.0105, 0.0117, 0.0098, 0.0123, 0.0106, 0.0128, 0.0121, 0.0161,
        0.0097, 0.0134, 0.0124, 0.0127, 0.0086, 0.0139, 0.0136, 0.0131, 0.0149,
        0.0092, 0.0135, 0.0115, 0.0143, 0.0128, 0.0106, 0.0136, 0.0125, 0.0135,
        0.0145, 0.0099, 0.0079, 0.0132, 0.0138, 0.0113, 0.0159, 0.0128, 0.0115,
        0.0107, 0.0145, 0.0093, 0.0138, 0.0111, 0.0123, 0.0140, 0.0157, 0.0168,
        0.0121, 0.0142, 0.0103, 0.0112, 0.0105, 0.0148, 0.0135, 0.0138, 0.0168,
        0.0112, 0.0095, 0.0095, 0.0141, 0.0112, 0.0140, 0.0098, 0.0120, 0.0116,
        0.0108, 0.0097, 0.0144, 0.0134, 0.0157, 0.0089, 0.0088, 0.0178, 0.0136],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [37]:
y

tensor([47, 30, 21, 23, 24, 37, 24, 60, 50, 31, 41, 47, 47, 35, 13, 44, 65, 48,
        58, 66, 51, 10, 67, 19, 10, 57, 70, 34, 40, 47, 33, 17, 59, 14, 68, 57,
        30, 30,  3, 14, 68, 23, 46, 68, 67, 46, 18, 52, 13, 29, 79, 24, 76, 42,
        52, 61,  2, 22, 43, 11, 30, 52, 22, 30])

In [40]:
loss

tensor(4.3945, device='cuda:0', grad_fn=<NllLossBackward0>)

In [15]:
X_train.shape

(10000, 1, 9, 9)

In [17]:
Y_train.shape

(10000, 81)

In [18]:
y_.shape

torch.Size([16, 81])

In [27]:
a = testset[2][1].argmax(dim=-1)

In [28]:
a

tensor(72)

In [20]:
Y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [21]:
T = Y_train.ToTensor()

TypeError: argmax() got an unexpected keyword argument 'dim'

In [27]:
data = torch.Tensor([[[
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0]
]]])

In [24]:
data.shape

torch.Size([1, 1, 5, 5])

In [25]:
# data = torch.Tensor(1, 64, 5, 5)
conv = nn.MaxPool2d(kernel_size=2)
out = conv(data)
out.shape

torch.Size([1, 1, 2, 2])

In [26]:
out

tensor([[[[0., 0.],
          [0., 1.]]]])

In [28]:
# inputs = torch.Tensor(1, 64, 5, 5)
conv = nn.MaxPool2d(kernel_size=2, stride=1)
out = conv(data)
out.shape

torch.Size([1, 1, 4, 4])

In [29]:
out

tensor([[[[0., 0., 0., 0.],
          [0., 1., 1., 0.],
          [0., 1., 1., 0.],
          [0., 0., 0., 0.]]]])

In [10]:
torch.save(model.state_dict(), "../models/MCTSAgentCNN.pt")

In [36]:
trainset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ../data/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )

In [39]:
print(model)

MNISTDNN(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (BN1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)


In [41]:
loss_fn

CrossEntropyLoss()

In [47]:
for x, y in train_loader:
    print(y)

tensor([8, 1, 7, 8, 4, 2, 6, 8, 2, 4, 8, 7, 5, 1, 1, 3, 4, 5, 9, 9, 3, 5, 5, 6,
        9, 7, 2, 9, 5, 0, 2, 2, 0, 4, 1, 8, 7, 7, 5, 2, 8, 9, 3, 1, 6, 9, 0, 2,
        6, 9, 1, 8, 0, 5, 2, 4, 6, 9, 4, 5, 8, 8, 1, 4, 4, 4, 9, 3, 9, 2, 7, 6,
        1, 3, 2, 5, 6, 5, 0, 0, 7, 7, 2, 7, 7, 1, 3, 6, 5, 3, 1, 0, 5, 0, 1, 9,
        8, 7, 1, 3, 0, 7, 7, 2, 8, 6, 3, 1, 9, 6, 6, 0, 9, 4, 8, 6, 0, 4, 1, 8,
        4, 5, 0, 6, 3, 6, 3, 4, 4, 8, 6, 1, 1, 6, 2, 6, 4, 4, 4, 2, 1, 0, 1, 2,
        3, 0, 4, 0, 4, 1, 1, 6, 3, 8, 2, 9, 7, 2, 9, 2, 2, 4, 5, 8, 7, 0, 8, 8,
        3, 2, 1, 8, 6, 5, 5, 3, 3, 7, 3, 5, 6, 4, 7, 5, 1, 8, 5, 0, 3, 8, 0, 3,
        6, 4, 0, 9, 1, 8, 8, 5, 8, 1, 5, 3, 3, 0, 5, 7, 0, 7, 3, 8, 2, 1, 3, 8,
        8, 1, 7, 3, 9, 4, 0, 1, 0, 3, 2, 5, 9, 2, 7, 7, 6, 3, 1, 5, 7, 7, 2, 5,
        8, 4, 0, 0, 9, 8, 4, 6, 3, 4, 6, 8, 3, 9, 3, 8])
tensor([0, 8, 0, 7, 9, 5, 4, 1, 2, 7, 1, 5, 5, 6, 9, 6, 1, 5, 4, 1, 4, 9, 6, 3,
        0, 3, 5, 8, 8, 6, 0, 3, 3, 6, 1, 0, 9, 2, 4, 6, 3, 4, 2

In [48]:
loss

tensor(1.4887, device='cuda:0', grad_fn=<NllLossBackward0>)

In [49]:
y_

tensor([[1.6208e-10, 1.5124e-15, 1.0000e+00,  ..., 1.0318e-10, 6.3672e-11,
         7.9840e-10],
        [1.9651e-16, 3.8407e-15, 1.6260e-19,  ..., 8.6929e-11, 6.5466e-14,
         4.8835e-11],
        [8.6161e-09, 2.1384e-12, 4.1127e-10,  ..., 5.5734e-11, 3.5426e-08,
         4.6580e-07],
        ...,
        [1.3633e-13, 1.0000e+00, 1.4438e-11,  ..., 2.5472e-10, 2.6938e-06,
         1.3435e-08],
        [2.3329e-13, 5.6320e-16, 5.0565e-10,  ..., 8.0373e-17, 8.9455e-11,
         1.0936e-12],
        [7.0781e-13, 7.8860e-14, 1.2475e-15,  ..., 6.6871e-10, 1.5389e-10,
         8.9596e-06]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [50]:
y

tensor([9, 5, 3, 3, 4, 6, 5, 5, 6, 6, 1, 3, 3, 5, 1, 6, 7, 2, 9, 0, 3, 1, 1, 9,
        2, 2, 7, 4, 8, 5, 7, 5, 7, 1, 2, 7, 2, 6, 7, 7, 0, 5, 4, 4, 3, 8, 8, 3,
        6, 0, 8, 9, 7, 0, 9, 5, 3, 9, 1, 1, 6, 0, 7, 4, 9, 1, 4, 3, 6, 1, 9, 7,
        7, 1, 3, 6, 1, 0, 6, 5, 9, 8, 8, 9, 7, 4, 0, 1, 4, 1, 2, 6, 7, 8, 0, 4])

In [51]:
oh_y = F.one_hot(y, num_classes=y_.shape[1]).float()
oh_y

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],


In [41]:
import torch
target = torch.tensor([[[[0.2, 0.5, 0.3], [0.3, 0.2, 0.5]]]])

In [42]:
a, b = torch.max(target, dim=-1)
print(a, b)

tensor([[[0.5000, 0.5000]]]) tensor([[[1, 2]]])


In [3]:
target.argmax(dim=-1)

tensor([[[1, 2]]])

In [16]:


X = np.load('../generated_games/features-200.npy')
Y = np.load('../generated_games/labels-200.npy')

In [18]:
Y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])